In [1]:
import pandas as pd

patient_path = '../brca_tcga_pub2015/data_clinical_patient.txt'
patient_df = pd.read_csv(patient_path, sep='\t', index_col=0)

In [2]:
survival_list = ['Overall Survival Status',
       'Overall Survival (Months)', 'Disease Free Status',
       'Disease Free (Months)']

survival_status = patient_df[survival_list]
surv_sta = survival_status.iloc[4:,:1]
surv_length = survival_status.iloc[4:,1:2]
surv_sta.rename(columns={surv_sta.columns[0]: 'surv_status'}, inplace=True)
surv_length.rename(columns={surv_length.columns[0]: 'surv_length'}, inplace=True)

drop_indices = surv_length[surv_length['surv_length'].isin(['[Not Available]', '[Discrepancy]'])].index
surv_length = surv_length.drop(index=drop_indices)

In [3]:
exp_path = '../cancer_data/2015_bulk_rna_seq.txt'
exp = pd.read_csv(exp_path, sep='\t', index_col=0)
exp.index = [i[:-3] for i in exp.index]

In [4]:
common_index = exp.index.intersection(surv_length.index)
surv_length = surv_length.loc[common_index]
exp = exp.loc[common_index]

In [5]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

X = exp.values
y = surv_length['surv_length'].values
y = np.array(y, dtype=np.float32)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler_X = MinMaxScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

# 转换为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)  # 确保 y 是二维的
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [7]:
import torch.nn as nn
import torch.optim as optim

class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_dim = X_train.shape[1]
model = RegressionModel(input_dim)

criterion = nn.MSELoss()  # 均方误差损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [8]:
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        # 前向传播
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 打印训练损失
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/500], Loss: 0.0065
Epoch [20/500], Loss: 0.0019


KeyboardInterrupt: 

In [ ]:
import numpy as np
from scipy.stats import pearsonr

# 确保模型处于评估模式
model.eval()
with torch.no_grad():
    y_pred = []
    y_true = []
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        y_pred.append(outputs.numpy())
        y_true.append(batch_y.numpy())
    
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    
    # 计算均方误差 (MSE)
    mse = np.mean((y_true - y_pred) ** 2)
    
    # 计算 R-squared
    r2 = 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))
    
    # 计算皮尔逊相关系数
    correlation, _ = pearsonr(y_true.flatten(), y_pred.flatten())
    
    print(f"Mean Squared Error: {mse}")
    print(f"R-squared: {r2}")
    print(f"Pearson Correlation Coefficient: {correlation}")
